# Notebook for Charged, Dipolar Fibrils

_M. Lund, May 2021_

![Alt text](fibril.png "linear fibril build from dipolar spheres representing proteins")

This notebook is a working document for building films made up of linear protein fibrils. Our aim is to define several ways of constructing fibrils and calculate the electric potential or field at arbitrary points.

In [1]:
import numpy as np
from copy import deepcopy

### Protein properties

Here we set the protein properties used to generate the fibrils. At the moment, a protein is defined by three parameters:

- Radius
- Net-charge
- Molecular dipole moment

In [2]:
radius = 15          # protein radius, Å
charge = 2           # protein net-charge, units of e
dipole_scalar = 150  # protein dipole moment, Debye
length = 10000       # fibril length, Å
fictitious_charge = 10.0 # used to generate explicit charges in dipoles

proteins_per_fibril = round(length / (2*radius))
global_position_storage = [] # storaged for generated point charges and their positions

### Helper functions

The following functions are used to generate fibrils which as currently straight lines of proteins placed in contact with
each other. The dipole moment is modelled _explicitly_, meaning that two extra charges are placed inside a single bead.
This makes it easier to calculate the electric potential, as we can operate with point charges only, and thus avoid the extra
complication of dealing with point dipoles.

The orientation of the protein dipole moment can be specified, but is currently pointing in the direction of the fibril.
The fibril is save to a PQR files which is simular to PDB, but carries information about the radius and charge as well.
PQR files can be viewed with for example VMD.

In [3]:
def makeAtomEntry(counter, pos, charge, radius):
    ''' generates a single ATOM entry for a PQR file '''
    atom_cnt = counter
    residue_cnt = counter
    atomname = "PRT"
    resname = "PTR"
    chain = "A"
    pqrformat = "{:6s}{:5d} {:^4s}{:1s}{:3s} {:1s}{:4d}{:1s}   {:8.3f}{:8.3f}{:8.3f}{:6.2f}{:6.2f}\n"
    global_position_storage.append(deepcopy([pos, charge, radius]))
    return pqrformat.format("ATOM", atom_cnt, atomname, "A", resname, chain, residue_cnt, "0",
                            pos[0], pos[1], pos[2], charge, radius)

def makeDipoleEntry(counter, position, dipole_direction):
    '''
    generates a particle consisting of three beads: neutral; positive, and negative. The first has
    a radius, while the other two represent a dipole and are placed inside the first particle. This
    allows for later calculation of the electric field. The two latter particles are displaced symmetrically
    around the first pointing in the `dipole_direction`.
    '''
    s = makeAtomEntry(counter, position, charge=charge, radius=radius)

    dipole_direction = dipole_direction / np.linalg.norm(dipole_direction)
    debye_to_electron_angstrom = 0.2081943
    displacement = debye_to_electron_angstrom * dipole_scalar * dipole_direction / (2.0 * fictitious_charge)
    assert np.linalg.norm(displacement) < radius, "increase `fictitious_charge`"
    s += makeAtomEntry(counter+1, position + displacement, charge=fictitious_charge, radius=1.0)
    s += makeAtomEntry(counter+2, position - displacement, charge=-fictitious_charge, radius=1.0)

    # check to see if the explicit dipole matches the input value
    mu = (position + displacement) * fictitious_charge + (position - displacement)*(-fictitious_charge)
    np.testing.assert_almost_equal(np.linalg.norm(mu), dipole_scalar * debye_to_electron_angstrom)
    return s

def makeMonopoleFibril(number_of_proteins, initial_position, direction):
    '''
    Make a single fibril where proteins lie up against each other in a line, growing in the `direction`
    
    '''
    displacement = 2.0 * radius * direction / np.linalg.norm(direction)
    s = ''
    position = initial_position[:]
    for counter in range(number_of_proteins):
        s += makeAtomEntry(counter+1, position, charge, radius)
        position += displacement
    return s

def makeDipoleFibril(number_of_proteins, initial_position, direction):
    '''
    Make a single fibril where proteins lie up against each other in a line, growing in the `direction`    
    '''
    displacement = 2.0 * radius * direction / np.linalg.norm(direction)
    s = ''
    position = deepcopy(initial_position)
    for counter in range(number_of_proteins):
        s += makeDipoleEntry(counter+1, position, direction)
        position += displacement
    return s

### Example

This will generate a simple fibril where the proteins are lined up next to each other, and with their dipole moments
pointing in the direction of the fibril. A snapshot of this is shown at the beginning of this notebook.

In [4]:
global_position_storage = []         # make sure this is empty before starting!
position = np.array([0.0, 0.0, 0.0]) # starting point for the fibril
fibril_direction = np.array([1.0, 0.0, 0.0]) # growing direction

print("proteins per fibril = ", proteins_per_fibril)
filename = 'fibrils.pqr'
with open(filename, 'w') as file:
    s = makeDipoleFibril(number_of_proteins=proteins_per_fibril, initial_position=position, direction=fibril_direction)
    file.write(s)

proteins per fibril =  333


## Calculation of Electric Potential and Field

In the operations above, we store all generated point charges in `global_position_storage` and we now use this to calculate
the electric potential at an arbitrary point.
Alternatively, we could later use [APBS](https://github.com/Electrostatics/apbs) which is a Poisson-Boltzmann solver commonly used in protein systems.

In [5]:
def calcPotential(target_position):
    ''' calculates the electric potential at a target position '''
    print('Number of charge points = ', len(global_position_storage))
    bjerrum_length = 7 * 80 # ~ vacuum
    potential = 0.0
    for pos, point_charge, radius in global_position_storage: # maybe slow; convert to numpy ops
        distance = np.linalg.norm(target_position - pos)
        potential +=  point_charge / distance
    return bjerrum_length * potential # kT/e

In [6]:
target_position = np.array([0.0, -10, 0.0]) # calculate potential here
print('potential at target = {} kT/e'.format(calcPotential(target_position)))

Number of charge points =  999
potential at target = 319.1623268342322 kT/e


## TODO List

- [x] Generation of single fibrils of arbitrary length and direction
- [x] Represent protein as spherical particle that can carry a charge and/or dipole moment
- [x] Routine for calculation of electric potential at arbitrary point
- [ ] Routine for calculation of electric field at arbitrary point
- [ ] Generate film
- [ ] Generate random structure